<a href="https://colab.research.google.com/github/doudoukiss/Airbnb/blob/main/kar1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
words = open('names.txt', 'r').read().splitlines()
#words[:10]

In [ ]:
#en(words)

In [ ]:
#min(len(w) for w in words)

In [ ]:
#max(len(w) for w in words)

In [ ]:
b = {}
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2 in zip(chs, chs[1:]):
    bigram = (ch1, ch2)
    b[bigram] = b.get(bigram, 0) + 1

In [ ]:
#sorted(b.items(), key = lambda kv: -kv[1])[:10]

In [ ]:
import torch

In [ ]:
N = torch.zeros((27, 27), dtype = torch.int32)

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
#stoi['<S>'] = 26
#stoi['<E>'] = 27
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [ ]:
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plt.figure(figsize = (16,16))
# plt.imshow(N, cmap='Blues')
# for i in range(27):
#   for j in range(27):
#     chstr = itos[i] + itos[j]
#     plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
#     plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
# plt.axis('off')

In [ ]:
# p = N[0,:].float()
# p = p / p.sum()
P = (N+1).float()
P /= P.sum(1, keepdim = True)

In [ ]:
P.shape

torch.Size([27, 27])

In [ ]:
g = torch.Generator().manual_seed(202220222022)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

'i'

In [ ]:
g = torch.Generator().manual_seed(202220222022)

for i in range(5):
  out = []
  ix = 0
  while True:
    #p = N[ix].float()
    #p = p / p.sum()
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement= True, generator=g).item()
    out.append(itos[ix])
    #print(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

ileyazevalyleet.
pedral.
gemen.
ahahenuawon.
day.


In [ ]:
log_likelihood = 0.0
n = 0

for w in words:
#for w in ["lilin"]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood = }')
nll = - log_likelihood
print(f'{nll/n=}')

log_likelihood = tensor(-559951.5625)
nll/n=tensor(2.4544)


In [ ]:
# create the training set of bigrams (x, y)
xs, ys = [], []

for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)


In [ ]:
#plt.imshow(xenc)

In [ ]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
# forward pass
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for the next character
loss = - probs[torch.arange(5), ys].log().mean()

In [ ]:
print(loss.item())

3.6891677379608154


In [ ]:
# backward pass
W.grad = None # set to zero the gradient
loss.backward()

In [ ]:
W.data += -0.1 * W.grad

In [ ]:
W.grad.shape

torch.Size([27, 27])

In [ ]:
# Optimization

In [ ]:
# create the dataset
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

number of examples:  228146


In [ ]:
# gradient descent
import torch.nn.functional as F
for k in range(10):

  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for the next character
  loss = - probs[torch.arange(num), ys].log().mean()
  print(loss.item())

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -50 * W.grad

2.4553630352020264
2.4553613662719727
2.455359935760498
2.4553580284118652
2.4553563594818115
2.455355405807495
2.4553537368774414
2.4553520679473877
2.455350637435913
2.4553492069244385


In [ ]:
nlls = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('------------')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the next correct chracter:', p.item())
  logp = torch.log(p)
  print('log likelihood', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())


------------
bigram example 1: .e (indexes 0,5)
input to the neural net: 0
label (actual next character): 5
probability assigned by the net to the next correct chracter: 0.01228625513613224
log likelihood -4.399273872375488
negative log likelihood: 4.399273872375488
------------
bigram example 2: em (indexes 5,13)
input to the neural net: 5
label (actual next character): 13
probability assigned by the net to the next correct chracter: 0.018050700426101685
log likelihood -4.014570713043213
negative log likelihood: 4.014570713043213
------------
bigram example 3: mm (indexes 13,13)
input to the neural net: 13
label (actual next character): 13
probability assigned by the net to the next correct chracter: 0.026691533625125885
log likelihood -3.623408794403076
negative log likelihood: 3.623408794403076
------------
bigram example 4: ma (indexes 13,1)
input to the neural net: 13
label (actual next character): 1
probability assigned by the net to the next correct chracter: 0.07367686182260513

In [ ]:
probs.shape

torch.Size([5, 27])

In [ ]:
loss

tensor(3.7693, grad_fn=<NegBackward0>)